In [67]:
import matplotlib.pyplot as plt
import torch
import torchvision
import torchvision.transforms.functional as F
import torchvision.transforms as T
from torchvision.io import ImageReadMode, read_image
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, random_split
import torch.optim as optim

In [68]:
# Set the path to your dataset
dataset_path = "../../pics/2750/"

In [70]:
transform = T.Compose([
    T.ToTensor(),
    T.RandomHorizontalFlip()
])

In [71]:
# Load the dataset
dataset = ImageFolder(dataset_path, transform=transform)

In [72]:
# Create a subset of the dataset containing forest and industrial classes
forest_industrial_dataset = torch.utils.data.Subset(dataset, [1, 4])

In [73]:
# Split the dataset into train and validation sets
train_ratio = 0.8
train_size = int(train_ratio * len(forest_industrial_dataset))
val_size = len(forest_industrial_dataset) - train_size
train_dataset, val_dataset = random_split(forest_industrial_dataset, [train_size, val_size])

# Create data loaders
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [74]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # Adjusted input size for linear layer
        self.drop1 = nn.Dropout(p=0.2)
        self.fc2 = nn.Linear(120, 84)
        self.drop2 = nn.Dropout(p=0.2)
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)  # Reshape to a 1-dimensional tensor
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

# Create an instance of the CNN model
model = CNN()

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Set the number of epochs
num_epochs = 20


In [75]:
# Lists to store the training loss and validation loss for each epoch
train_losses = []
val_losses = []

# Training loop
for epoch in range(num_epochs):
    # Training
    model.train()
    train_loss = 0.0
    for images, labels in train_dataloader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    avg_train_loss = train_loss / len(train_dataloader)
    train_losses.append(avg_train_loss)
    
    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for images, labels in val_dataloader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    avg_val_loss = val_loss / len(val_dataloader)
    val_losses.append(avg_val_loss)
    
    # Print the loss and epoch
    print(f"Epoch: {epoch+1}/{num_epochs} | Training Loss: {avg_train_loss:.4f} | Validation Loss: {avg_val_loss:.4f}")

# Truncate the losses if the training loop was interrupted
if len(train_losses) > num_epochs:
    train_losses = train_losses[:num_epochs]
if len(val_losses) > num_epochs:
    val_losses = val_losses[:num_epochs]

# Plot the losses
fig, ax = plt.subplots(figsize=(8, 6))
ax.set(xlabel='Epoch', ylabel='Loss', title='Loss Function')
ax.plot(range(1, len(train_losses) + 1), train_losses, color='blue', linewidth=2, marker='o', label='Training Loss')
ax.plot(range(1, len(val_losses) + 1), val_losses, color='red', linewidth=2, marker='o', label='Validation Loss')
ax.legend()
ax.grid(True)
plt.show()



RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x2704 and 400x120)

In [ ]:
# Calculate accuracy on the validation dataset
correct = 0
total = 0
model.eval()  # Set the model to evaluation mode

with torch.no_grad():
    for images, labels in val_dataloader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Accuracy on the validation dataset: {accuracy:.2%}')
